# Thai Wiki Language Model for Text Generation

This notebook details how you can use pretrained language model on [Thai Wikipedia Dump](https://dumps.wikimedia.org/thwiki/latest/thwiki-latest-pages-articles.xml.bz2) to generate texts.

In [0]:
# #uncomment if you are running from google colab
# !pip install pythainlp
# !pip install fastai
# !pip install emoji

In [0]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

import matplotlib.pyplot as plt
import seaborn as sns

import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

from pythainlp.ulmfit import *

In [0]:
try:
  THWIKI_LSTM = _THWIKI_LSTM
except:
  THWIKI_LSTM = THWIKI_LSTM

In [7]:
#get dummy data
imdb = untar_data(URLs.IMDB_SAMPLE)
dummy_df = pd.read_csv(imdb/'texts.csv')

#get vocab
thwiki_itos = pickle.load(open(THWIKI_LSTM['itos_fname'], 'rb'))
thwiki_vocab = fastai.text.transform.Vocab(thwiki_itos)

#dummy databunch
tt = Tokenizer(tok_func=ThaiTokenizer, lang='th', pre_rules=pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=thwiki_vocab, max_vocab=60000, min_freq=3)]
data_lm = (TextList.from_df(dummy_df, imdb, cols=['text'], processor=processor)
    .split_by_rand_pct(0.2)
    .label_for_lm()
    .databunch(bs=64))

data_lm.sanity_check()

In [8]:
#check vocab size
len(data_lm.vocab.itos)

60005

In [9]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load_pretrained(**THWIKI_LSTM)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (800 items)
x: LMTextList
xxbos   this   is   a   xxunk   xxunk   film .   the   acting ,   script   and   xxunk   are   all   xxunk .   the   music   is   good ,   too ,   though   it   is   mostly   early   in   the   film ,   when   things   are   still   xxunk   xxunk .   there   are   no   really   superstars   in   the   cast ,   though   several   faces   will   be   xxunk .   the   entire   cast   does   an   excellent   job   with   the   script .   
   
   but   it   is   hard   to   watch ,   because   there   is   no   good   end   to   a   situation   like   the   one   presented .   it   is   now   xxunk   to   xxunk   the   british   for   setting   xxunk   and   xxunk   against   each   other ,   and   then   xxunk   xxunk   them   into   two   countries .   there   is   some   merit   in   this   view ,   but   it ' s   also   true   that   no   one   forced   xxunk   and   xxunk   in   the   region   to   xxunk  

In [10]:
print(learn.predict('กาลครั้งหนึ่งนานมาแล้ว ', 200, temperature=0.8, min_p=0.005, sep=''))

กาลครั้งหนึ่งนานมาแล้ว 
 
 ช่วงสองหนุ่มที่มีฐานะร่ำรวย และเป็นลูกสาวของ ดร. นิโคลัส (รับบทโดย อลิสัน คิง) เป็นลูกชายคนโตของ นาย โอลิเวอร์ และ มารดา นิโคลัส ในช่วงแรก เขาเป็นพ่อของ โอลลี เขามีพ่อ และแม่ เป็นคนที่มีจิตใจอ่อนโยน และเป็นเพื่อนสนิทกับ ดร. คาลิส (โรส วิลสัน) ที่เป็นพ่อของเธอนั่นเอง 
 
 ในปี ค.ศ. 1967 สมิธได้รับการเลี้ยงดูจาก จอร์จ สปีลเบิร์ก และ ซาร่า ซึ่งต่อมาก็ได้รับการดูแลจากแม่ ซึ่งทำให้เขามีทักษะด้านวิทยาศาสตร์และเทคโนโลยีสูง หลังจากสำเร็จการศึกษาจากมหาวิทยาลัยแล้ว โรส ได้เดินทางไปที่ ประเทศออสเตรเลีย เพื่อเป็นที่ปรึกษา และในช่วงนั้นเอง ที่ได้พบกับ คริส เจ โรส ผู้เป็นมารดา
